### Melon 100 chart 스크래핑
* 100곡의 노래제목, ID를 추출하기
* Song ID로 상세페이지를 100번 요청해서 곡의 상세정보를 추출
* 100곡의 상세정보를 json file에 저장하기
* json file을 Pandas의 DataFrame 객체로 저장해서 표데이터를 만들기
* DataFrame 객체를 DB의 테이블로 저장하기

In [1]:
import requests
from bs4 import BeautifulSoup
import re

url = 'https://www.melon.com/chart/index.htm'
req_header = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'
}

res = requests.get(url, headers=req_header)
print(res.ok)
if res.ok:
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
#     print(len(soup.select("a[href*='playSong']"))) #100
#     print(len(soup.select("div.wrap_song_info a[href*='playSong']"))) #100
#     print(len(soup.select("tr div.wrap_song_info a[href*='playSong']"))) #100
    
    #100곡의 song 정보를 저장할 list 선언
    song_list = []
    song_atags = soup.select("tr div.wrap_song_info a[href*='playSong']")
    for idx, atag in enumerate(song_atags,1):
        #1곡의 song 정보를 저장할 dict 선언
        song_dict = {}
        song_title = atag.text
        href = atag['href']
        song_id = None
        matched = re.search(r'(\d+)\);',href)
        if matched:
            #print(matched.group(0), matched.group(1))
            song_id = matched.group(1)
            
        song_url = f'https://www.melon.com/song/detail.htm?songId={song_id}'
        
        song_dict['title']  = song_title
        song_dict['id'] = song_id
        song_dict['url'] = song_url
        #print(song_dict)
        song_list.append(song_dict)

print(len(song_list))        
song_list[0:3]

True
100


[{'title': '쉬어 (Feat. MINO) (Prod. GRAY)',
  'id': '34215822',
  'url': 'https://www.melon.com/song/detail.htm?songId=34215822'},
 {'title': 'Wake Up (Prod. 코드 쿤스트)',
  'id': '34215821',
  'url': 'https://www.melon.com/song/detail.htm?songId=34215821'},
 {'title': 'strawberry moon',
  'id': '34101563',
  'url': 'https://www.melon.com/song/detail.htm?songId=34101563'}]

#### 100곡의 노래 상세정보 추출하기

In [23]:
import requests
from bs4 import BeautifulSoup
import re

req_header = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'
}

song_detail_list = []
for idx, song in enumerate(song_list,1):
    res = requests.get(song['url'], headers=req_header)
    soup = BeautifulSoup(res.text, 'html.parser')
    if res.ok:
        song_detail_dict = {}
        print(idx, song['title'])
        song_detail_dict['곡명'] = song['title']
        singer = soup.select("a[href*='.goArtistDetail'] span")
        if singer:
            song_detail_dict['가수'] = singer[0].text
        
        meta = soup.select("div.meta dd")    
        if meta:
            song_detail_dict['앨범'] = meta[0].text
            song_detail_dict['발매일'] = meta[1].text
            song_detail_dict['장르'] = meta[2].text
                
        #좋아요건수
        song_id = song['id']
        songlike_url = f'https://www.melon.com/commonlike/getSongLike.json?contsIds={song_id}'
        songlike_res = requests.get(songlike_url, headers=req_header)
        if songlike_res.ok:
            song_detail_dict['좋아요'] = songlike_res.json()['contsLike'][0]['SUMMCNT']
            
        #가사
        if soup.select('div#d_video_summary'):
            lyric_str = soup.select('div#d_video_summary')[0].text
            regexp = re.compile(r'[\n\r\t\']')
            lyric = regexp.sub("",lyric_str.strip())
        else:
            lyric = ''
        song_detail_dict['가사'] = lyric
        
        song_detail_list.append(song_detail_dict)

print(len(song_detail_list))
song_detail_list[0:3]

1 쉬어 (Feat. MINO) (Prod. GRAY)
2 Wake Up (Prod. 코드 쿤스트)
3 strawberry moon
4 사랑은 늘 도망가
5 Savage
6 TROUBLE (Prod. Slom)
7 STAY
8 신호등
9 다정히 내 이름을 부르면
10 너와 나의 Memories
11 My Universe
12 Next Level
13 이제 나만 믿어요
14 너를 생각해
15 OHAYO MY NIGHT
16 Permission to Dance
17 낙하 (with 아이유)
18 Butter
19 바라만 본다
20 별빛 같은 나의 사랑아
21 그대라는 사치
22 다시 사랑한다면 (김필 Ver.)
23 Weekend
24 HERO
25 잊었니
26 Dynamite
27 끝사랑
28 Bad Habits
29 Bk Love
30 Peaches (Feat. Daniel Caesar & Giveon)
31 헤픈 우연
32 고백
33 Rock with you
34 어제 너는 나를 버렸어
35 라일락
36 시간을 거슬러 (낮에 뜨는 달 X 케이윌)
37 찰나가 영원이 될 때 (The Eternal Moment)
38 좋아좋아
39 Celebrity
40 계단말고 엘리베이터
41 나 그댈위해 시 한편을 쓰겠어
42 DUMB DUMB
43 Queendom
44 모래시계
45 롤린 (Rollin')
46 사이렌 Remix (Feat. UNEDUCATED KID, Paul Blanco)
47 가까이 있어서 몰랐어
48 비가 오는 날엔 (2021)
49 내 손을 잡아
50 별거 없던 그 하루로
51 어떻게 이별까지 사랑하겠어, 널 사랑하는 거지
52 작은 것들을 위한 시 (Boy With Luv) (Feat. Halsey)
53 Dun Dun Dance
54 밤하늘의 별을(2020)
55 밝게 빛나는 별이 되어 비춰줄게
56 비와 당신
57 Savage Love (Laxed - Siren Beat) (BTS Remix)
58 가을 타나 봐
59 너는 내 세상이었어
60

[{'곡명': '쉬어 (Feat. MINO) (Prod. GRAY)',
  '가수': '아넌딜라이트(Anandelight)',
  '앨범': '쇼미더머니 10 Episode 1',
  '발매일': '2021.11.06',
  '장르': '랩/힙합',
  '좋아요': 51080,
  '가사': '오늘은 눈 좀 붙이고 쉬어굳어진 몸을 풀어차가운 목걸이가 조여숨이 잘 안 쉬어지잖아꿈꾸러 가자 좀 쉬어 쉬어집 앞 공사판 숨을 쉬어밤이 되면 맘의 눈을 떠관중들이 내는 환호성보이지 않지만 꿈을 꿨어 yeah어린 나는 이제 춤을 춰2014의 불확실함에벌벌 떨던 꼬마에게 말을 거네 yeah"숨을 쉬어"숨을 들이쉬고 내쉬고를 반복다시 넘어지더라도 일어나길 반복결국 끝에 가선 푸른 풀밭이 날 반겨쉴 만한 물가에서 비파와 수금으로 반격 yeah이제 그냥 켜 봐라 TV마술사 아닌데 NOW YOU SEE ME거기서 외칠 말은 단 하나다른 거 없다 마, 다만 하나님의 사랑그저 편히 쉬어 너와 나의 mood소리 높여서 숨을 내쉬어Just one more time한 번 더 준비 Im ready숨을 크게 들이쉬고 나서Let it burn Let it burnImma show you how I did itYall get it?난 못 쉬어서 한숨 쉬어한숨 쉬어 가려 나온 게 show많은 게 쉬워져 버려 아쉬워대체 우리 언제 시원한 공기 마셔1 for the show 2 for the money not her8 mile 속의 에미넴이 말했어기회는 한번 쥐어진 microphoneYo 첫 마디에 D.O.G we on it나 서울로 다시 올라왔어 그때 그 건방진 녀석언제 이리 컸어 관심 없어서 잘 몰랐어So who that unofficialboyy 봤어이수린 thats my name wassupWassup 나왔어 방송 why 돈 벌러 왔어다시 돌아왔어 돌고 돌고 돌아서다시 돌아왔어 돌고 돌고 돌아그물, 덫, 발사대기, 포획하러 showShow me the money, ackermann또다시 돌아왔어

In [25]:
import json

with open('data/songs.json','w',encoding='utf-8') as file:
    json.dump(song_detail_list, file)